# Assignment 3: Composition with Convolutional Maps
Use the following code to experiment with convolving different samples. When you find an interesting creation, save it to use in your assignment by clicking on the three dots in the sample player and selecting "Download".

In [1]:
!pip install librosa
import matplotlib.pyplot as plt 
from ipywidgets import widgets
from IPython.display import Audio, display, clear_output
import librosa
import librosa.display
import numpy as np
from pathlib import Path

SR = 44100

def next_power_of_2(x):  
    return 1 if x == 0 else 2**(x - 1).bit_length()

## Listen to some samples
Use the dropdown below to listen to the different samples

In [2]:
sample_list = [str(file.name) for file in Path('./samples').iterdir() if file.is_file()]

sample_dropdown = widgets.Dropdown(
    options=sample_list,
    description="Sample:"
)

# Create a button widget
listen_button = widgets.Button(description="Listen")

# Create an Output widget to display the generated music
listen_output_widget = widgets.Output()

# Define a function to be called when the button is clicked
def on_listen_button_click(b):
    with listen_output_widget:
        clear_output(wait=True)
        path = Path('./samples') / sample_dropdown.value
        x, _ = librosa.load(path, sr=SR)
        display(Audio(x, rate=SR))

# Attach the function to the button's click event
listen_button.on_click(on_listen_button_click)

# Display the widgets and button
widgets.VBox([sample_dropdown, listen_button, listen_output_widget])


## Convolve two samples
Use the following code to convolve two samples together. You can use the output of this in your assignment.

With sound files we are in the time domain which enables us to use the Convolution Theorem. The theorem states that convolution in the time domain is the same as complex multiplication in the frequency domain. In other words, multiplying the frequency content (spectra) of two signals is the same as performing convolution. So, 

$$ y(t) = x(t) * h(t) = IFFT(X(k)H(k)) $$ 

where $X(k)$ and $H(k)$ are frequency representations of the signals $x$ and $h$, and $y$ is our convolved signal.

In [3]:
sample_list = [str(file.name) for file in Path('./samples').iterdir() if file.is_file()]

def convolve(x, h):
    # the FFT is most efficient when the length of the signal is a power of two
    if x.size > h.size:
        N = next_power_of_2(x.size)
    else:
        N = next_power_of_2(h.size)

    # calculate the real part of the FFT for each signal 
    x_fft = np.fft.rfft(x, N)
    h_fft = np.fft.rfft(h, N)

    # multiply the two signals
    convolved = x_fft * h_fft

    # use IFFT to convert to time domain signal
    y = np.fft.irfft(convolved)

    return y


# Create Dropdown widgets for the two samples
x_dropdown = widgets.Dropdown(
    options=sample_list,
    description="Sample 1:",  
)

h_dropdown = widgets.Dropdown(
    options=sample_list,
    description="Sample 2:"
)

# Create a button widget
convolve_button = widgets.Button(description="Convolve")

# Create an Output widget to display the generated music
convolve_output_widget = widgets.Output()

# Define a function to be called when the button is clicked
def on_convolve_button_click(b):
    with convolve_output_widget:
        clear_output(wait=True)  # Clear the output widget without clearing the dropdowns
        print(f"Convolving {x_dropdown.value} and {h_dropdown.value}")
        path1 = Path('./samples') / x_dropdown.value
        path2 = Path('./samples') / h_dropdown.value
        x, _ = librosa.load(path1, sr=SR)
        h, _ = librosa.load(path2, sr=SR)
        y = convolve(x, h)
        print("Convolution complete")
        display(Audio(y, rate=SR))

# Attach the function to the button's click event
convolve_button.on_click(on_convolve_button_click)

# Display the widgets and button
widgets.VBox([x_dropdown, h_dropdown, convolve_button, convolve_output_widget])